In [ ]:
# !pip install faiss-cpu
#!pip install sentence_transformers

In [2]:
import pandas as pd
import numpy as np

In [3]:
sales = pd.read_excel("./data.xlsx")  # data.xlsx
fcst  = pd.read_excel("./Forcast.xlsx")  # Forcast.xlsx

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678025 entries, 0 to 678024
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Site              678021 non-null  object 
 1   Date              678019 non-null  object 
 2   Invoice Number    678013 non-null  object 
 3   Customer Code     678002 non-null  object 
 4   Name              677996 non-null  object 
 5   Item Code         677993 non-null  object 
 6   Item Description  677983 non-null  object 
 7   Quantity          677916 non-null  float64
dtypes: float64(1), object(7)
memory usage: 41.4+ MB


In [5]:
fcst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BRAND                24 non-null     object 
 1   ACTIVE               24 non-null     object 
 2   Product              24 non-null     object 
 3   Product description  24 non-null     object 
 4   PROD                 24 non-null     object 
 5   TYP                  24 non-null     object 
 6   STATUS               24 non-null     object 
 7   ON HAND INITIAL      0 non-null      float64
 8   PO INITIAL           0 non-null      float64
 9   OCT                  18 non-null     float64
 10  NOV                  18 non-null     float64
 11  DEC                  24 non-null     int64  
 12  JAN                  24 non-null     int64  
 13  FEB                  23 non-null     float64
 14  MAR                  23 non-null     float64
 15  APR                  24 non-null     int64

In [6]:
# Keep only needed columns (robust to case/whitespace)
def pick(df, names):
    cols = {c.strip().lower(): c for c in df.columns}
    out = {}
    for n in names:
        key = n.strip().lower()
        if key in cols: out[n] = cols[key]
    return df[[out[n] for n in out]]

sales = pick(sales, ["Site","Date","Invoice Number","Customer Code","Name","Item Code","Item Description","Quantity"])
fcst  = pick(fcst , ["BRAND","ACTIVE","Product","Product description","PROD","TYP","STATUS"]) \
        .join(pd.read_excel("./Forcast.xlsx").drop(columns=["BRAND","ACTIVE","Product","Product description","PROD","TYP","STATUS"], errors="ignore"))

In [8]:
fcst.columns

Index(['BRAND', 'ACTIVE', 'Product', 'Product description', 'PROD', 'TYP',
       'STATUS', 'ON HAND INITIAL', 'PO INITIAL', 'OCT', 'NOV', 'DEC', 'JAN',
       'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT.1',
       'NOV.1', 'DEC.1', 'JAN.1', 'FEB.1', 'MAR.1', 'APR.1', 'MAY.1', 'JUN.1',
       'JULY', 'AUG.1', 'SEPT.1', 'OCT.2', 'NOV.2', 'DEC.2', 'JAN.2', 'FEB.2',
       'MAR.2', 'APR.2', 'MAY.2', 'JUN.2', 'JUL.1', 'AUG.2', 'SEPT.2', 'OCT.3',
       'NOV.3', 'DEC.3', 'JAN.3', 'FEB.3', 'MAR.3', 'APR.3', 'MAY.3', 'JUN.3',
       'JUL.2', 'AUG.3', 'SEPT.3', 'OCT.4', 'NOV.4', 'DEC.4', 'JAN.4', 'FEB.4',
       'MAR.4', 'APR.4', 'MAY.4', 'JUN.4', 'JUL.3', 'AUG.4', 'SEP', 'OCT.5',
       'NOV.5', 'DEC.5'],
      dtype='object')

In [24]:
# EDA + Data Sanity + Linkage - 
# Sets of IDs to compare (case-insensitive)
sales["item_code_std"] = sales["Item Code"].astype(str).str.upper().str.strip()
fcst["product_std"]    = fcst["Product"].astype(str).str.upper().str.strip()

# --- Find exact matches ---
direct_codes = sorted(set(sales["item_code_std"]) & set(fcst["product_std"]))
print("Direct matches:", len(direct_codes))
print(len(sales["item_code_std"]))
print(len(fcst["product_std"]))

Direct matches: 8
678025
24


In [ ]:
# All forcast products - 8 are present in sales, but not all, hence forcast for missing products in Sales will 
# be missing.

In [18]:
neg_rows = sales[sales["Quantity"]<0]
print(len(sales), len(neg_rows))


678025 12611


In [42]:
# Building linkage table - 
link = pd.DataFrame({"item_code_std": list(direct_codes)})
link = link.merge(fcst[["product_std"]].drop_duplicates(),
                  left_on="item_code_std", right_on="product_std", how="left")
link = link.rename(columns={"product_std":"product_link"})
assert not link["item_code_std"].duplicated().any(), "Duplicate key in linkage table!"


In [31]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   item_code_std  8 non-null      object
 1   product_link   8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes


In [33]:
# --- Check primary key uniqueness ---
assert not link["item_code_std"].duplicated().any(), "Duplicate key in linkage table!"

In [35]:
# Apply linkage -
before = len(sales)
sales_linked = sales.merge(link[["item_code_std","product_link"]], on="item_code_std", how="left")
after  = len(sales_linked)
assert before == after, "Row count changed after merge!"

sales_linked["forecast_available"] = np.where(sales_linked["product_link"].notna(), "Yes", "No")
sales_linked["Order Type"] = np.where(sales_linked["Quantity"] < 0, "Cancelled", "Accepted")
print("Rows preserved:", after)
print(sales_linked.head(3)[["Item Code","product_link","forecast_available","Quantity","Order Type"]])


Rows preserved: 678025
              Item Code product_link forecast_available  Quantity Order Type
0     NBOMOATS16X350GPB          NaN                 No       2.0   Accepted
1            HISTV43A4K          NaN                 No      -1.0  Cancelled
2  NBOMMULTIGR16X350GPB          NaN                 No      15.0   Accepted


In [36]:
sales_linked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678025 entries, 0 to 678024
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Site                678021 non-null  object 
 1   Date                678019 non-null  object 
 2   Invoice Number      678013 non-null  object 
 3   Customer Code       678002 non-null  object 
 4   Name                677996 non-null  object 
 5   Item Code           677993 non-null  object 
 6   Item Description    677983 non-null  object 
 7   Quantity            677916 non-null  float64
 8   item_code_std       678025 non-null  object 
 9   product_link        42867 non-null   object 
 10  forecast_available  678025 non-null  object 
 11  Order Type          678025 non-null  object 
dtypes: float64(1), object(11)
memory usage: 62.1+ MB


In [38]:
sales_linked['forecast_available'].unique()

array(['No', 'Yes'], dtype=object)

In [41]:
# I have assumed rightmost months as 2025 months and 2024, 2023 and so on as we traverse data leftwards - 

# Rename forecast month columns per your rule:
# - From the LEFTMOST columns up to DEC: treat as 2025 months.
# - First JAN from the LEFT = 2026 JAN (and onward months = 2026)
# - Second JAN from the LEFT = 2027 JAN (and onward months = 2027)
# - And so on.

# Assumes `fcst` is already a pandas DataFrame loaded from Forcast.xlsx

month_abbr = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
def is_month_col(c):
    return isinstance(c, str) and c.strip().upper()[:3] in month_abbr

month_cols = [c for c in fcst.columns if is_month_col(c)]

jan_count = 0
rename_map = {}
for col in month_cols:  # left-to-right
    m3 = col.strip().upper()[:3]
    if m3 == "JAN":
        jan_count += 1           # 1st JAN => 2026, 2nd JAN => 2027, etc.
    year = 2025 + jan_count      # pre-1st JAN (jan_count=0) => 2025
    rename_map[col] = f"{m3}_{year}"

fcst_renamed = fcst.rename(columns=rename_map)

# (optional) quick check
print({k: rename_map[k] for k in month_cols[:12]})
print(fcst_renamed.columns)

{'OCT': 'OCT_2025', 'NOV': 'NOV_2025', 'DEC': 'DEC_2025', 'JAN': 'JAN_2026', 'FEB': 'FEB_2026', 'MAR': 'MAR_2026', 'APR': 'APR_2026', 'MAY': 'MAY_2026', 'JUN': 'JUN_2026', 'JUL': 'JUL_2026', 'AUG': 'AUG_2026', 'SEPT': 'SEP_2026'}
Index(['BRAND', 'ACTIVE', 'Product', 'Product description', 'PROD', 'TYP',
       'STATUS', 'ON HAND INITIAL', 'PO INITIAL', 'OCT_2025', 'NOV_2025',
       'DEC_2025', 'JAN_2026', 'FEB_2026', 'MAR_2026', 'APR_2026', 'MAY_2026',
       'JUN_2026', 'JUL_2026', 'AUG_2026', 'SEP_2026', 'OCT_2026', 'NOV_2026',
       'DEC_2026', 'JAN_2027', 'FEB_2027', 'MAR_2027', 'APR_2027', 'MAY_2027',
       'JUN_2027', 'JUL_2027', 'AUG_2027', 'SEP_2027', 'OCT_2027', 'NOV_2027',
       'DEC_2027', 'JAN_2028', 'FEB_2028', 'MAR_2028', 'APR_2028', 'MAY_2028',
       'JUN_2028', 'JUL_2028', 'AUG_2028', 'SEP_2028', 'OCT_2028', 'NOV_2028',
       'DEC_2028', 'JAN_2029', 'FEB_2029', 'MAR_2029', 'APR_2029', 'MAY_2029',
       'JUN_2029', 'JUL_2029', 'AUG_2029', 'SEP_2029', 'OCT_2029', '

In [43]:
# 2) Keep only forecast columns we want (product key, description, month-year)
import re
month_col = re.compile(r"^(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)_\d{4}$", re.I)
keep_cols = ["product_std", "Product description"] + [c for c in fcst_renamed.columns if month_col.match(str(c))]
fcst_slim = fcst_renamed[keep_cols].drop_duplicates(subset=["product_std"])

# 3) Merge (left) — preserves sales row count; no duplication
before = len(sales_linked)
enriched = sales_linked.merge(fcst_slim, left_on="product_link", right_on="product_std", how="left")
assert len(enriched) == before, "Row count changed — check keys!"


In [47]:
enriched.shape

(678025, 77)

In [46]:
enriched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678025 entries, 0 to 678024
Data columns (total 77 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Site                 678021 non-null  object 
 1   Date                 678019 non-null  object 
 2   Invoice Number       678013 non-null  object 
 3   Customer Code        678002 non-null  object 
 4   Name                 677996 non-null  object 
 5   Item Code            677993 non-null  object 
 6   Item Description     677983 non-null  object 
 7   Quantity             677916 non-null  float64
 8   item_code_std        678025 non-null  object 
 9   product_link         42867 non-null   object 
 10  forecast_available   678025 non-null  object 
 11  Order Type           678025 non-null  object 
 12  product_std          42867 non-null   object 
 13  Product description  42867 non-null   object 
 14  OCT_2025             42095 non-null   float64
 15  NOV_2025         

In [50]:
# Prepping parquet file for simpler chunking - 
# Convert all object columns to string to avoid ArrowTypeError
for col in enriched.select_dtypes(include="object").columns:
    enriched[col] = enriched[col].astype(str)

enriched.to_parquet("enriched.parquet", index=False)

In [51]:
# Prepare data for chunking - 

merged = pd.read_parquet("enriched.parquet")
merged["Date"] = pd.to_datetime(merged["Date"], errors="coerce")
merged["month_key"] = merged["Date"].dt.strftime("%b_%Y").str.upper()
print(merged.shape)
print(merged[["Date","Item Code","Quantity","month_key"]].head(5))

(678025, 78)
        Date             Item Code  Quantity month_key
0 2024-01-02     NBOMOATS16X350GPB       2.0  JAN_2024
1 2024-01-02            HISTV43A4K      -1.0  JAN_2024
2 2024-01-02  NBOMMULTIGR16X350GPB      15.0  JAN_2024
3 2024-01-02  LILYULTRASOFTNR10X12       1.0  JAN_2024
4 2024-01-02  NBOMMBANAPL16X350GPB      10.0  JAN_2024


In [ ]:
# Building text chunks + metadata =>

# Keep only rows where forecast product exists (8 SKUs)
subset = merged[merged["product_std"].notna()].copy()
print("Subset shape:", subset.shape)

# find forecast month columns like JAN_2025, FEB_2026, ...
month_re = re.compile(r"^(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)_(\d{4})$", re.I)
month_cols = [c for c in subset.columns if month_re.match(str(c))]

# group by month abbrev (choose the earliest year, e.g., 2025)
from collections import defaultdict
month_groups = defaultdict(list)
for c in month_cols:
    m, y = month_re.match(c).groups()
    month_groups[m.upper()].append((int(y), c))
for m in month_groups:
    month_groups[m].sort()  # ascending by year

def pick_forecast_any_year(row):
    # use month name only (ignore sales year)
    mk = str(row.get("month_key",""))
    mon = mk.split("_")[0] if "_" in mk else None
    if mon and mon.upper() in month_groups:
        # pick earliest year for that month (e.g., JAN_2025)
        return row.get(month_groups[mon.upper()][0][1])
    return np.nan

def row_to_chunk(row):
    date_str = pd.to_datetime(row["Date"], errors="coerce")
    date_str = date_str.date().isoformat() if pd.notna(date_str) else "NA"
    fcst_val = pick_forecast_any_year(row)
    fcst_txt = "N/A" if pd.isna(fcst_val) else str(fcst_val)
    return {
        "text": (
            f"Product: {row.get('Item Code')}; "
            f"Site: {row.get('Site')}; "
            f"Date: {date_str}; "
            f"Month: {row.get('month_key')}; "
            f"Sales Qty: {row.get('Quantity')}; "
            f"Forecast: {fcst_txt}; "
            f"Order Type: {row.get('Order Type','Accepted')}"
        ),
        "item_code": row.get("Item Code"),
        "site": row.get("Site"),
        "date": date_str,
        "month_key": row.get("month_key"),
        "source": "merged.parquet"
    }

chunks = pd.DataFrame([row_to_chunk(r) for _, r in merged.iterrows()])
chunks.to_parquet("chunks.parquet", index=False)
print("chunks rebuilt:", chunks.shape)

# save for embedding step - 
chunks.to_parquet("chunks.parquet", index=False)


chunks rebuilt: (678025, 6)


In [ ]:
# # Embeddings (MiniLM, CPU) → FAISS index + metadata

# import faiss
# from sentence_transformers import SentenceTransformer # type: ignore

# # 1) Load chunks
# chunks = pd.read_parquet("chunks.parquet")

# # 2) Encode text → vectors (384-dim, CPU)
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # small & fast on CPU
# emb = model.encode(chunks["text"].tolist(), batch_size=256, show_progress_bar=True, normalize_embeddings=True)
# emb = np.asarray(emb, dtype="float32")

# # 3) FAISS index (cosine via inner product on normalized vectors)
# index = faiss.IndexFlatIP(emb.shape[1])
# index.add(emb)

# # 4) Save index + metadata
# faiss.write_index(index, "faiss.index")
# chunks.reset_index(drop=True).to_parquet("meta.parquet", index=False)

# print("Indexed:", index.ntotal, "vectors; dim:", emb.shape[1])


Batches:   0%|          | 0/2649 [00:00<?, ?it/s]

Indexed: 678025 vectors; dim: 384


In [69]:
print("num_chunks:", len(chunks))
chunks1 = chunks.drop_duplicates("text")
print("num_chunks:", len(chunks1))
chunks["text"] = chunks["text"].str.slice(0, 800)

num_chunks: 678025
num_chunks: 473670


In [70]:
# 4) Embeddings - 
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
emb = model.encode(chunks1["text"].tolist(), batch_size=512,
                   show_progress_bar=True, normalize_embeddings=True).astype("float32")

# 5) FAISS index
index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb)
faiss.write_index(index, "faiss_forecast.index")
chunks1.to_parquet("meta_forecast.parquet", index=False)

print("FAISS index ready for forecast subset:", index.ntotal)

Batches:   0%|          | 0/926 [00:00<?, ?it/s]

FAISS index ready for forecast subset: 473670


In [ ]:
# # Retriever: embed query → search FAISS → return top-k chunks + metadata (CPU-only)

# import faiss, numpy as np, pandas as pd
# from sentence_transformers import SentenceTransformer

# # 1) Load FAISS + metadata (from 2.3)
# index = faiss.read_index("faiss.index")
# meta  = pd.read_parquet("meta.parquet")  # contains columns: text, item_code, site, month_key, source, ...

# # 2) Load encoder (same as used for indexing)
# enc = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# def retrieve(query: str, k: int = 5):
#     # 3) Encode query (normalize for cosine/IP)
#     q = enc.encode([query], normalize_embeddings=True)
#     q = np.asarray(q, dtype="float32")
#     # 4) Search
#     scores, idx = index.search(q, k)         # scores shape: (1,k), idx shape: (1,k)
#     idx = idx[0].tolist(); scores = scores[0].tolist()
#     # 5) Pack results with metadata
#     out = meta.iloc[idx].copy()
#     out["score"] = scores
#     return out.reset_index(drop=True)

In [71]:
# Forecast-only retriever: uses faiss_forecast.index + meta_forecast.parquet

import faiss, numpy as np, pandas as pd
from sentence_transformers import SentenceTransformer

# 1) Load index + metadata (built for the 8 forecast SKUs)
index = faiss.read_index("faiss_forecast.index")
meta  = pd.read_parquet("meta_forecast.parquet")  # columns: text, item_code, site, month_key, source

# 2) Load the same encoder used during indexing
enc = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_forecast(query: str, k: int = 5) -> pd.DataFrame:
    # Encode query (normalize for cosine/IP)
    q = enc.encode([query], normalize_embeddings=True)
    q = np.asarray(q, dtype="float32")
    # Search
    scores, idx = index.search(q, k)
    idx = idx[0].tolist(); scores = scores[0].tolist()
    # Pack results
    out = meta.iloc[idx].copy()
    out["score"] = scores
    return out.reset_index(drop=True)


In [72]:
# # Tiny test:
q = "Compare sales vs forecast for HISSPL10HPINV in JAN 2025"
print(retrieve_forecast(q, k=5)[["score","item_code","site","month_key","text"]])

      score      item_code  site month_key  \
0  0.608307  HISSPL10HPINV  BBEN  FEB_2024   
1  0.605483  HISSPL10HPINV  BAS1  FEB_2024   
2  0.604496  HISSPL10HPINV  BAS1  FEB_2024   
3  0.603777  HISSPL10HPINV  BAS1  FEB_2024   
4  0.603150  HISSPL10HPINV  MIKE  JUN_2024   

                                                text  
0  Product: HISSPL10HPINV; Site: BBEN; Date: 2024...  
1  Product: HISSPL10HPINV; Site: BAS1; Date: 2024...  
2  Product: HISSPL10HPINV; Site: BAS1; Date: 2024...  
3  Product: HISSPL10HPINV; Site: BAS1; Date: 2024...  
4  Product: HISSPL10HPINV; Site: MIKE; Date: 2024...  


In [73]:
# Prompt builder -

import requests

# 1) Build context from retrieved rows (add simple citations)
def build_context(rows):
    lines = []
    for i, r in rows.iterrows():
        cite = f"[CIT{i+1}: {r.get('item_code','NA')} | {r.get('site','NA')} | {r.get('month_key','NA')}]"
        lines.append(f"{cite} {r['text']}")
    return "\n".join(lines), [f"CIT{i+1}" for i in range(len(rows))]

In [74]:
# 2) Ollama generator - Ask Ollama with strict grounding
def ask_ollama(question, rows, model="llama3.2:latest", max_tokens=250, temperature=0.2):
    context, cites = build_context(rows)
    prompt = (
        "SYSTEM: Answer ONLY using the CONTEXT. If info is missing, say 'Not available'. "
        "Be concise and include citation IDs (e.g., [CIT1]).\n\n"
        f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:"
    )
    r = requests.post("http://localhost:11434/api/generate", json={
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": max_tokens,
        "stream": False
    }, timeout=120)
    return r.json()["response"]

In [75]:
# --- tiny example flow ---
q = "Compare sales vs forecast for HISSPL10HPINV in JAN_2027."
topk = retrieve_forecast(q, k=5)
print(ask_ollama(q, topk))

Not available.


2025-08-29 10:31:21.473 
  command:

    streamlit run c:\Anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-29 10:31:21.475 Session state does not function when running a script without `streamlit run`


In [80]:
import requests
print(requests.post("http://127.0.0.1:11434/api/generate",
                    json={"model":"llama3.2:latest","prompt":"ping","stream":False}).json())


{'model': 'llama3.2:latest', 'created_at': '2025-08-29T05:10:54.7626003Z', 'response': '*ping*', 'done': True, 'done_reason': 'stop', 'context': [128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 271, 128009, 128006, 882, 128007, 271, 10194, 128009, 128006, 78191, 128007, 271, 9, 10194, 9], 'total_duration': 6205370600, 'load_duration': 4886905400, 'prompt_eval_count': 26, 'prompt_eval_duration': 1017000000, 'eval_count': 4, 'eval_duration': 281000000}


In [ ]:
# Run:   streamlit run app.py
# Needs: ollama serve  (and model pulled), faiss_forecast.index, meta_forecast.parquet in working dir